# install

In [1]:
#install the package, on colab make sure to upgrade existing packages. This is not needed in a clean env.
#! pip install --upgrade deepforest albumentations pyyaml

In [2]:
# pip install git+https://github.com/weecology/DeepForest.git

# load libraries

In [ ]:
#load the modules
import os
import time
import numpy as np
from deepforest import main 
from deepforest import get_data
from deepforest import utilities
from deepforest import preprocess


# load example image

In [ ]:
#convert hand annotations from xml into retinanet format
#The get_data function is only needed when fetching sample package data
YELL_xml = get_data("2019_YELL_2_528000_4978000_image_crop2.xml")
annotation = utilities.xml_to_annotations(YELL_xml)
annotation.head()

,image_path,xmin,ymin,xmax,ymax,label
0,2019_YELL_2_528000_4978000_image_crop2.png,681,369,699,386,Tree
1,2019_YELL_2_528000_4978000_image_crop2.png,980,59,998,79,Tree
2,2019_YELL_2_528000_4978000_image_crop2.png,953,135,964,150,Tree
3,2019_YELL_2_528000_4978000_image_crop2.png,969,183,995,208,Tree
4,2019_YELL_2_528000_4978000_image_crop2.png,742,623,751,635,Tree


In [ ]:
#load the image file corresponding to the annotaion file
YELL_train = get_data("2019_YELL_2_528000_4978000_image_crop2.png")
image_path = os.path.dirname(YELL_train)
#Write converted dataframe to file. Saved alongside the images
annotation.to_csv(os.path.join(image_path,"train_example.csv"), index=False)

## Prepare Training and Validation Data
  - 75% Training Data
  - 25% Validation Data

In [ ]:
#Find annotation path
annotation_path = os.path.join(image_path,"train_example.csv")
#crop images will save in a newly created directory
#os.mkdir(os.getcwd(),'train_data_folder')
crop_dir = os.path.join(os.getcwd(),'train_data_folder')
train_annotations= preprocess.split_raster(path_to_raster=YELL_train,
                                 annotations_file=annotation_path,
                                 base_dir=crop_dir,
                                 patch_size=400,
                                 patch_overlap=0.05)

/usr/local/lib/python3.7/dist-packages/rasterio/__init__.py:220: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/usr/loca

In [ ]:
#Split image crops into training and test. Normally these would be different tiles! Just as an example.
image_paths = train_annotations.image_path.unique()
#split 25% validation annotation
valid_paths = np.random.choice(image_paths, int(len(image_paths)*0.25) )
valid_annotations = train_annotations.loc[train_annotations.image_path.isin(valid_paths)]
train_annotations = train_annotations.loc[~train_annotations.image_path.isin(valid_paths)]

In [ ]:
#View output
train_annotations.head()
print("There are {} training crown annotations".format(train_annotations.shape[0]))
print("There are {} test crown annotations".format(valid_annotations.shape[0]))

#save to file and create the file dir
annotations_file= os.path.join(crop_dir,"train.csv")
validation_file= os.path.join(crop_dir,"valid.csv")
#Write window annotations file without a header row, same location as the "base_dir" above.
train_annotations.to_csv(annotations_file,index=False)
valid_annotations.to_csv(validation_file,index=False)

There are 569 training crown annotations
There are 205 test crown annotations


In [ ]:
annotations_file

## Training & Evaluating Using CPU

In [ ]:
#initial the model and change the corresponding config file
m = main.deepforest()
m.config["train"]["csv_file"] = annotations_file
m.config["train"]["root_dir"] = os.path.dirname(annotations_file)
#Since this is a demo example and we aren't training for long, only show the higher quality boxes
m.config["score_thresh"] = 0.4
m.config["train"]['epochs'] = 2
m.config["validation"]["csv_file"] = validation_file
m.config["validation"]["root_dir"] = os.path.dirname(validation_file)
#create a pytorch lighting trainer used to training 
m.create_trainer()
#load the lastest release model 
m.use_release()

Reading config file: /usr/local/lib/python3.7/dist-packages/deepforest/data/deepforest_config.yml


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:148: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=False)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=False)`.
  f"Setting `Trainer(checkpoint_callback={checkpoint_callback})` is deprecated in v1.5 and will "
GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:1567: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  "GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`."


Model from DeepForest release https://github.com/weecology/DeepForest/releases/tag/1.0.0 was already downloaded. Loading model from file.
Loading pre-built model: https://github.com/weecology/DeepForest/releases/tag/1.0.0


In [ ]:
start_time = time.time()
m.trainer.fit(m)
print(f"--- Training on CPU: {(time.time() - start_time):.2f} seconds ---")


  | Name  | Type      | Params
------------------------------------
0 | model | RetinaNet | 32.1 M
------------------------------------
31.9 M    Trainable params
222 K     Non-trainable params
32.1 M    Total params
128.592   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

ValueError: ignored

In [ ]:
m.get_transform

AttributeError: ignored

In [ ]:
#create a directory to save the predict image
save_dir = os.path.join(os.getcwd(),'pred_result')
try:
  os.mkdir(save_dir)
except FileExistsError:
  pass
results = m.evaluate(annotations_file, os.path.dirname(annotations_file), iou_threshold = 0.4, savedir = save_dir)

## Training & Evaluating Using GPU

In [ ]:
#initial the model and change the corresponding config file
m = main.deepforest()
m.config['gpus'] = '-1' #move to GPU and use all the GPU resources
m.config["train"]["csv_file"] = annotations_file
m.config["train"]["root_dir"] = os.path.dirname(annotations_file)
m.config["score_thresh"] = 0.4
m.config["train"]['epochs'] = 2
m.config["validation"]["csv_file"] = validation_file
m.config["validation"]["root_dir"] = os.path.dirname(validation_file)
#create a pytorch lighting trainer used to training 
m.create_trainer()
#load the lastest release model 
m.use_release()

Reading config file: /usr/local/lib/python3.7/dist-packages/deepforest/data/deepforest_config.yml


GPU available: True, used: True
TPU available: False, using: 0 TPU cores


Model from DeepForest release https://github.com/weecology/DeepForest/releases/tag/1.0.0 was already downloaded. Loading model from file.
Loading pre-built model: https://github.com/weecology/DeepForest/releases/tag/1.0.0


In [ ]:
start_time = time.time()
m.trainer.fit(m)
print(f"--- Training on GPU: {(time.time() - start_time):.2f} seconds ---")

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type      | Params
------------------------------------
0 | model | RetinaNet | 32.1 M
------------------------------------
31.9 M    Trainable params
222 K     Non-trainable params
32.1 M    Total params
128.592   Total estimated model params size (MB)



--- Training on GPU: 40.67 seconds ---


In [ ]:
#save the prediction result to a prediction folder
save_dir = os.path.join(os.getcwd(),'pred_result')
try:
  os.mkdir(save_dir)
except FileExistsError:
  pass
results = m.evaluate(annotations_file, os.path.dirname(annotations_file), iou_threshold = 0.4, savedir= save_dir)

In [ ]:
results

{'box_precision': 0.6005210661419024,
 'box_recall': 0.828532593751304,
 'class_recall':    label  recall  precision  size
 0      0     1.0        1.0   628,
 'results':    prediction_id  ...  match
 0              0  ...   True
 11            11  ...  False
 10            10  ...   True
 7              7  ...   True
 8              8  ...   True
 ..           ...  ...    ...
 10            10  ...   True
 1              1  ...   True
 3              3  ...   True
 4              4  ...   True
 0              0  ...   True
 
 [634 rows x 7 columns]}

In [ ]:
results['box_precision']

0.6005210661419024

In [ ]:
results["box_recall"]

0.828532593751304

In [ ]:
results["results"]

,prediction_id,truth_id,IoU,predicted_label,true_label,image_path,match
0,0,0,0.685860,Tree,Tree,2019_YELL_2_528000_4978000_image_crop2_1.png,True
11,11,0,0.021521,Tree,Tree,2019_YELL_2_528000_4978000_image_crop2_10.png,False
10,10,1,0.823035,Tree,Tree,2019_YELL_2_528000_4978000_image_crop2_10.png,True
7,7,2,0.823054,Tree,Tree,2019_YELL_2_528000_4978000_image_crop2_10.png,True
8,8,3,0.827481,Tree,Tree,2019_YELL_2_528000_4978000_image_crop2_10.png,True
...,...,...,...,...,...,...,...
10,10,5,0.813103,Tree,Tree,2019_YELL_2_528000_4978000_image_crop2_9.png,True
1,1,6,0.654108,Tree,Tree,2019_YELL_2_528000_4978000_image_crop2_9.png,True
3,3,7,0.698520,Tree,Tree,2019_YELL_2_528000_4978000_image_crop2_9.png,True
4,4,8,0.806653,Tree,Tree,2019_YELL_2_528000_4978000_image_crop2_9.png,True


In [ ]:
results["class_recall"]

,label,recall,precision,size
0,0,1.0,1.0,628
